<a href="https://colab.research.google.com/github/nicolez9911/colab/blob/main/AdvML_L2_S1_N1_Solution_ImplementingOneVersusAll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# One versus All (OvA)

This notebook provides a stub for implementing the One versus All approach.


### Imports


In [ ]:
import numpy as np


### Adapted Perceptron Class

In order to use the class for the OvA approach we make three changes:

* fit method now returns a weight vector
* net_input expects a weight vector as parameter
* predict method expects a weight vector as parameter

In [ ]:
class Perceptron(object):

    """Perceptron binary classifier.

    Parameters
    -----------
    eta : float
       Learning rate (value between 0.0 and 1.0)
    epochs  : int
       Number of passes over the training set

    Attributes
    -----------


    """

    def __init__(self, eta=0.01, epochs=10, random_state=1):
        self.eta = eta
        self.epochs = epochs
        self.random_state = random_state

    def fit(self, X, y):
        """ Fit training data; i.e. train the Perceptron

        Parameters
        -----------
        X : dataframe with shape - [n_samples, n_features]
          feature vectors, where n_samples is the number of samples
          in the data set, n_features is the number of features
        y : dataframe with shape = [n_samples].
            1 target value per sample (the label).

        Returns
        -----------
        self : weight vector w

        """

        # this initialize the weights
        rgen = np.random.RandomState(self.random_state)
        w = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])

        # this will be used to store the number of updates per epoch


        for _ in range(self.epochs):
            for xi, target in zip(X, y.T):
                update = self.eta * (target - self.predict(xi, w))
                w[1:] += update * xi
                w[0] += update
        return w

    def net_input(self, x, w):
        """Calculate net input for given sample vector and weight vector"""
        net_input = w[0] + sum(t[0] * t[1] for t in zip(x, w[1:]))
        return net_input

    def predict(self, x, w):
        """Predict based on computing the net input and the threshold function"""
        net_input = self.net_input(x, w)
        if net_input >= 0.0:
            prediction = 1
        elif net_input < 0.0:
            prediction = -1
        return prediction



### Import the IRIS dataset

The cell below imports the full IRIS dataset as contained in sklearn.   

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
y = iris.target
X = iris.data[:,:]

### Exercise 1: Analyse Transformation of Targets

Take a look at below method and compare it to the discussion we had of the OvA approach.
The method provides a key implementation piece necessary for implementing a very simple
OvA classifier based on our Perceptron implementation.



In [ ]:
import copy

def split_targets(y):
    # identify the number of classes in the dataset
    cn = np.size(np.unique(y))
    sub_t =  []
    for c in range(0,cn):
        # prepare temporary C vs notC sub-probem labels
        y_temp = copy.deepcopy(y)
        ind = np.argwhere(y_temp.astype(int) == c)
        ind = ind[:,0]
        ind2 = np.argwhere(y_temp.astype(int) != c)
        ind2 = ind2[:,0]
        #Check indices
        y_temp[[ind]] = 1
        y_temp[[ind2]] = -1
        sub_t.append(y_temp)
    return sub_t


### Exercise 2: Implement OvA classification

 1. Implement the most simplistic OvA implementation.
Using the split_targets method should make it much easier to do so.

 2. Test your trained weight vectors for each class with samples from the IRIS dataset.
    It suffices if you print the scores for the different weight vectors. We just want
    to get to the point where we can see the principle of OvA in action.
  


In [ ]:
# the number of classes. This should be useful for the implementation
cnum = np.size(np.unique(y))

# transform the target to be compatible with OvA
sub_targets = split_targets(y)

# train the C weight vectors (1 for each class)
ppn = Perceptron(eta=0.1, epochs=300)
weights = []
for c in range(0,cnum):
    temp_w = ppn.fit(X, sub_targets[c])
    weights.append(temp_w)


# test your implementation with multiple
x = X[2,:]
for c in range(0,cnum):
    print(ppn.net_input(x, weights[c]))


/home/dev/BIN/anaconda3/envs/deng_ml/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/dev/BIN/anaconda3/envs/deng_ml/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


-2.852823671031209
-7.4208236710301865
30.757176328968256


### Exercise 3: Train and Test with MNIST

Test your OvA implementation with the MINIST dataset.

1. Import the MNIST dataset
2. Train an OvA classifier for the full set of digits or a subset
3. Conduct some sanity checks by testing with number samples